# OEL TASK

In [57]:
%%writefile oel.cu
#include <cuda.h>
#include <stdio.h>
#include <random>
#include <chrono>
#include <iostream>

using namespace std;

mt19937 rng(chrono::steady_clock::now().time_since_epoch().count());
int rand(int lo, int hi) {
    if (lo > hi) swap(lo, hi);
    return lo + rng() % (hi - lo + 1);
} // returns in the range [lo, hi]

__constant__ int k = 3;
__constant__ float kernel[9];

__global__ void gaussian_blur(unsigned char *input, unsigned char *output, int n, int m){
  int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
  if(globalIdx < n * m){

    float sum = 0;
    for(int i = max(0, globalIdx / m - 1); i <= min(n - 1, globalIdx / m + 1); i++){
      for(int j = max(0, globalIdx % m - 1); j <= min(m - 1, globalIdx % m + 1); j++){
        int kx = (i - globalIdx / m) + 1;
        int ky = (j - globalIdx % m) + 1;
        sum += input[i * m + j] * kernel[kx * k + ky];
      }
    }

    output[globalIdx] = (unsigned char)min(max(int(sum / 16), 0), 255);
  }
}

__global__ void avg_blur(unsigned char *input, unsigned char *output, int n, int m){
  int globalIdx = blockIdx.x * blockDim.x + threadIdx.x;
  if(globalIdx < n * m){
    int count = 0;
    int sum = 0;
    for(int i = max(0, globalIdx / m - 1); i <= min(n - 1, globalIdx / m + 1); i++){
      for(int j = max(0, globalIdx % m - 1); j <= min(m - 1, globalIdx % m + 1); j++){
        sum += input[i * m + j];
        count++;
      }
    }

    output[globalIdx] = (unsigned char)min(max(sum / count, 0), 255);
  }
}


void print_matrix(float *matrix, int n, int m){
  for(int i = 0; i < n; i++){
    for(int j = 0; j < m; j++){
      cout << matrix[i * m + j] << " ";
    } cout << endl;
  }
}

void print_matrix(int *matrix, int n, int m){
  for(int i = 0; i < n; i++){
    for(int j = 0; j < m; j++){
      cout << (int) matrix[i * m + j] << " ";
    } cout << endl;
  }
}

void print_matrix(unsigned char *matrix, int n, int m){
  for(int i = 0; i < n; i++){
    for(int j = 0; j < m; j++){
      cout << (int) matrix[i * m + j] << " ";
    } cout << endl;
  }
}

int main(){
  int n = 3;
  int m = 3;
  int nxm = n * m;
  int size = nxm * sizeof(int);
  unsigned char *host_input = (unsigned char *) malloc(size);
  unsigned char *host_output = (unsigned char *) malloc(size);
  float host_filter[9] = {1,2,1,2,4,2,1,2,1};
  /*for(int i = 0; i < 3 ; i++){
    int v = 1;
    for(int j = 0; j < 3 ; j++){
      host_filter[i * 3 + j] = v;
      v--;
    }
  }*/

  for(int i = 0; i < nxm; i++){
    host_input[i] = rand(0,10);
  }

  unsigned char *device_input;
  unsigned char *device_output;
  cudaMalloc(&device_input, size);
  cudaMalloc(&device_output, size);
  cudaMemcpyToSymbol(kernel, host_filter, sizeof(host_filter));
  cudaMemcpy(device_input, host_input, size, cudaMemcpyHostToDevice);

  int blocksize = 256;
  int gridsize = (nxm + blocksize - 1) / blocksize;

  cout << "INPUT MATRIX: " << endl;
  print_matrix(host_input, n, m);

  cout << "GAUSSIAN FILTER: " << endl;
  print_matrix(host_filter, 3, 3);

  gaussian_blur<<<gridsize, blocksize>>>(device_input, device_output, n, m);
  cudaMemcpy(host_output, device_output, size, cudaMemcpyDeviceToHost);
  avg_blur<<<gridsize, blocksize>>>(device_output, device_input, n, m);

  cout << "AFTER GAUSSIAN FILTER: " << endl;
  print_matrix(host_output, n, m);

  cudaMemcpy(host_output, device_input, size, cudaMemcpyDeviceToHost);

  cout << "AFTER AVG BLUR FILTER: " << endl;
  print_matrix(host_output, n, m);
  return 0;
}

Overwriting oel.cu


In [58]:
!nvcc oel.cu -o oel
!./oel

INPUT MATRIX: 
3 3 8 
3 3 5 
2 2 6 
GAUSSIAN FILTER: 
1 2 1 
2 4 2 
1 2 1 
AFTER GAUSSIAN FILTER: 
1 3 3 
2 3 3 
1 2 2 
AFTER AVG BLUR FILTER: 
2 2 3 
2 2 2 
2 2 2 
